In [1]:
#pip install pulp

In [2]:
from pulp import * # importa-se a biblioteca pulp para obter uma solução para o modelo

### Projeto 1 - Escolha de Carteira de Investimentos [Modificado]

Uma empresa de investimentos X gerencia recursos de terceiros através da escolha de carteiras de investimento para diversos clientes, baseados em bonds de diversas empresas. Um de seus clientes exige que:

- Não mais de 25% do total aplicado deve ser investido em único investimento
- Um valor superior a 50% do total aplicado deve ser investido em títulos de maturidade maiores que dez anos.
- o Total Aplicado em títulos de alto risco deve ser, no máximo, de 50% do total investido.

In [38]:
# Esse comando é utilizado para o pandas reconhecer o formato .odf do excel: pip install odfpy 
import pandas as pd

df = pd.read_excel('Dados_Investimento.ods',engine='odf')
df

,Títulos,Retorno/Ano,Anos para Vencimento,Risco
0,A,0.087,7,2
1,B,0.095,11,3
2,C,0.120,12,5
3,D,0.090,14,1
4,E,0.135,3,4
5,F,0.211,15,1
6,G,0.165,19,3
7,H,0.078,13,2
8,I,0.151,2,4
9,J,0.099,10,2


#### Descrição das Variáveis

- Títulos -> Nome dos Títulos do conjunto de dados;
- Retorno/Ano -> Taxa de Retorno do Investimento no ano;
- Anos para Vencimento -> Valor máximo de anos 
- Risco -> Índice de Risco para cada título: (1- Muito baixo), (2- baixo), (3- regular), (4- alto), (5- muito alto) 


## Modelagem

$Maximizar: $ $$0.087\frac{(P1)}{100}+0.095\frac{(P2)}{100}+0.120\frac{(P3)}{100}+0.090\frac{(P4)}{100}+0.135\frac{(P5)}{100}+0.211\frac{(P6)}{100}+0.165\frac{(P7)}{100}+0.078\frac{(P8)}{100}+0.151\frac{(P9)}{100}+0.099\frac{(P10)}{100},$$

Sujeito a:

$\sum\limits_{i=1}^{10}P_i=100$ (Restrição de Orçamento do Cliente)

$P_i \leq 25, \forall i \in [1,10]$ (Restrição de Risco por título)

$P2+P3+P4+P6+P7+P8 \geq 50$ (Restrição de Títulos com vencimento superior a dez anos)

$P1+P5+P9+P10 \leq 50$ (Restrição de Títulos de alto risco)

$P_i \geq 0, \forall i \in [1,10]$ (Não-negatividade)


In [24]:
# Criando as variáveis de percentuais

prob = LpProblem("myProblem", LpMaximize)
P1 = LpVariable("P1", 0, 25)
P2 = LpVariable("P2", 0, 25)
P3 = LpVariable("P3", 0, 25)
P4 = LpVariable("P4", 0, 25)
P5 = LpVariable("P5", 0, 25)
P6 = LpVariable("P6", 0, 25)
P7 = LpVariable("P7", 0, 25)
P8 = LpVariable("P8", 0, 25)
P9 = LpVariable("P9", 0, 25)
P10 = LpVariable("P10", 0, 25)

# Função Objetivo
prob += 0.087*(P1*0.01) + 0.095*(P2*0.01) + 0.120*(P3*0.01) + 0.090*(P4*0.01) + 0.135*(P5*0.01) + 0.211*(P6*0.01) + 0.165*(P7*0.01) + 0.078*(P8*0.01) + 0.151*(P9*0.01) + 0.099*(P10*0.01) , "obj"

# Restrições

prob += P1 + P2 + P3 + P4 + P5 + P6 + P7 + P8 + P9 + P10 == 100, "c1"
prob += P2 + P3 + P4 + P6 + P7 + P8 >= 50, "c2"
prob += P1 + P5 + P9 + P10 <= 50, "c3"



In [25]:
status = prob.solve()
status

1

In [26]:
LpStatus[status]

'Optimal'

In [44]:
for v in prob.variables(): # esse laço percorre as variáveis definidas para o problema
    print(v.name, "=", v.varValue)

P1 = 0.0
P10 = 0.0
P2 = 0.0
P3 = 0.0
P4 = 0.0
P5 = 25.0
P6 = 25.0
P7 = 25.0
P8 = 0.0
P9 = 25.0


In [48]:
print("Taxa de retorno = ", value(prob.objective)) # valor da função objetivo

Taxa de retorno =  0.1655


#### De acordo com os requisitos solicitados pelo cliente, os investimentos seriam de 25% nos títulos E, F, G e I.

#### Referências

LACHTERMACHER, Gerson. Pesquisa Operacional na Tomada de Decisões. Rio de Janeiro: Editora Campus, 2002.